#### Movies ChatBot

This project is the development of a chatbot for movie queries/recommendations using the TMBDB API as a base

In [1]:
import requests

from langchain.agents import initialize_agent, Tool
from langchain.tools import StructuredTool
from langchain.agents import AgentType
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory

from langchain.prompts.chat import MessagesPlaceholder
from typing import Tuple, Dict

In [2]:
import requests
import os
import re

class TmdbApiWrapper:
    BASE_URL = "https://api.themoviedb.org/3"
    DEFAULT_LANGUAGE = "pt-BR"
    HEADERS = {
        "accept": "application/json",
        "Authorization": f"Bearer {os.getenv('BEARER_TOKEN_TMDB')}"
    }

    def __init__(self):
        pass

    def _make_request(self, url, additional_parameters=None):
        params = {"language": self.DEFAULT_LANGUAGE}

        if additional_parameters:
            params.update(additional_parameters)

        response = requests.get(url, headers=self.HEADERS, params=params)
        # response.raise_for_status()
        return response.json()
        
    # TODO: definir método de retorno do filme mais similar com a busca
    def search_movie(self, movie_title):
        """Search for a movie by its title."""
        
        url = f"{self.BASE_URL}/search/movie"
        params = {
            "query": movie_title,
            "page": 1
        }
        response = self._make_request(url, params)
        return response.get("results")[0] if response.get("results") else None

    def get_casting(self, movie_title):
        """Get the casting of a movie by its ID."""

        movie_id = self.search_movie(movie_title).get("id")

        url = f"{self.BASE_URL}/movie/{movie_id}/credits"

        response = self._make_request(url)
        all_cast = response.get("cast", [])
        actor_cast = [
            {"name": actor["name"], "character": actor["character"]}
            for actor in all_cast
            if actor.get("known_for_department") == "Acting"
        ]
        return f"Elenco do filme {movie_title}: {actor_cast[:10]}"

    def similar_movies(self, movie_title):
        """Get similar movies to a movie by its ID."""

        movie_id = self.search_movie(movie_title).get("id")

        url = f"{self.BASE_URL}/movie/{movie_id}/similar"
        params = {"page": 1}
        response = self._make_request(url, params)

        similar_movies = [movie.get("title") for movie in response.get("results", [])]
        return f"Esses são alguns filmes similares a {movie_title}: {similar_movies[:10]}"

    def popular_movies(self, time_window):
        """Get the most popular movies of the time window(day or week)"""

        time_window = re.sub(r"\n", "", time_window)

        url = f"{self.BASE_URL}/trending/movie/{time_window}"

        response = self._make_request(url)
        popular_movies = [movie["title"] for movie in response["results"]]
        
        return f"Os filmes populares no período escolhido são: {popular_movies}"

In [3]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()

class LLMConfig():
    """
    Contains the configuration of the LLM.
    """
    model = "llama3-8b-8192"
    groq_api_key = os.getenv("GROQ_API_KEY")
    llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name=model)

In [4]:
llm_config = LLMConfig()

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# from langchain.globals import set_debug

# set_debug(True)

def setup_agent() -> AgentExecutor:
    """
    Sets up the tools for a function based chain.
    """
    cfg = LLMConfig()
    imdb = TmdbApiWrapper()

    tools = [
        Tool(
            name = "Pesquisa de filmes",
            func=imdb.search_movie,
            description="Útil para buscar informações sobre filmes como sinopse, média de avaliações, data de lançamento, entre outros, incuindo seu ID na base do IMDB que poderá ser usado em outras tools."
        ),
        Tool(
            name = "Elenco do filme",
            func=imdb.get_casting,
            description="Retorna o elenco de um filme a partir de seu ID."
        ),
        Tool(
            name = "Filmes similares",
            func=imdb.similar_movies,
            description="Retorna filmes similares a partir do ID de um filme."
        ),
        Tool(
            name = "Filmes populares",
            func=imdb.popular_movies,
            description="Retorna os filmes mais populares de uma janela de tempo(use day ou week)."
        ),
    ]

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, k=3)

    return initialize_agent(
        tools, 
        cfg.llm, 
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
        verbose=True,
        max_iterations=3,
        memory=memory,
        
    )

In [6]:
agent_chain = setup_agent()

/home/cristianomendieta/.cache/pypoetry/virtualenvs/movies-chatbot-cyk_sWxh-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
PROMPT_MESSAGE = """
Você é um assistente que concede informações relevantes sobre filmes que o usário queira saber.\n
Você não possui conhecimento prévio sobre filmes e por isso, DEVE utilizar as tools a seguir e responder o usário com base nessas informações.\n
Responda o usuário em português.\n\n

TOOLS:\n
------\n\n

> Pesquisa de filmes: Útil para buscar informações sobre filmes como sinopse, média de avaliações no TMDB, data de lançamento e ID do filme na base do TMDB, entre outros.\n
> Elenco do filme: Retorna o elenco de um filme a partir de seu nome.\n
> Filmes similares: Retorna filmes similares a partir de seu nome.\n
> Filmes populares: Retorna os filmes mais populares de uma janela de tempo(use day ou week).\n\n

Para usar uma tool, por favor siga o seguinte formato:\n\n

To use a tool, please use the following format:\n

```\n
Thought: Do I need to use a tool? Yes\n
Action: the action to take, should be one of [Pesquisa de filmes, Elenco do filme, Filmes similares, Filmes populares]\n
Action Input: the input to the action\n
Observation: the result of the action\n
````\n\n

Quando você tiver uma resposta para dizer ao Humano, ou se não precisar usar uma tool, você DEVE usar o formato:\n\n

```
Thought: Do I need to use a tool? No\n
AI: [your response here]\n\n
```\n

Begin!\n

Previous conversation history:\n
{chat_history}\n

New input: {input}\n
{agent_scratchpad}
"""

In [8]:
agent_chain.agent.llm_chain.prompt.template = PROMPT_MESSAGE

In [9]:
agent_chain.run("Qual a data de lançamento do filme The Matrix?")

/home/cristianomendieta/.cache/pypoetry/virtualenvs/movies-chatbot-cyk_sWxh-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes

Action: Pesquisa de filmes

Action Input: The Matrix

Observation: {'adult': False, 'backdrop_path': '/icmmSD4vTTDKOq2vvdulafOGw93.jpg', 'genre_ids': [28, 878], 'id': 603, 'original_language': 'en', 'original_title': 'The Matrix', 'overview': 'Em um futuro próximo, Thomas Anderson (Keanu Reeves), um jovem programador de computador que mora em um cubículo escuro, é atormentado por estranhos pesadelos nos quais encontra-se conectado por cabos e contra sua vontade, em um imenso sistema de computadores do futuro. Em todas essas ocasiões, acorda gritando no exato momento em que os eletrodos estão para penetrar em seu cérebro. À medida que o sonho se repete, Anderson começa a ter dúvidas sobre a realidade. Por meio do encontro com os misteriosos Morpheus (Laurence Fishburne) e Trinity (Carrie-Anne Moss), Thomas descobre que é, assim como outras pessoas, vítima do Matrix, um sistema inteligente e artificial que man

'O filme The Matrix foi lançado em 31 de março de 1999.'

In [10]:
agent_chain.run("Qual o elenco do filme The Matrix?")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes

Action: Elenco do filme

Action Input: The Matrix

Observation: Elenco do filme The Matrix
: [{'name': 'Keanu Reeves', 'character': 'Neo'}, {'name': 'Laurence Fishburne', 'character': 'Morpheus'}, {'name': 'Carrie-Anne Moss', 'character': 'Trinity'}, {'name': 'Hugo Weaving', 'character': 'Agent Smith'}, {'name': 'Gloria Foster', 'character': 'Oracle'}, {'name': 'Joe Pantoliano', 'character': 'Cypher'}, {'name': 'Marcus Chong', 'character': 'Tank'}, {'name': 'Julian Arahanga', 'character': 'Apoc'}, {'name': 'Matt Doran', 'character': 'Mouse'}, {'name': 'Belinda McClory', 'character': 'Switch'}]
Thought:```
Thought: Do I need to use a tool? No

AI: O elenco do filme The Matrix é composto por Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, Gloria Foster, Joe Pantoliano, Marcus Chong, Julian Arahanga, Matt Doran e Belinda McClory, que interpretam respectivamente os personagens Neo, Morpheus

'O elenco do filme The Matrix é composto por Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, Gloria Foster, Joe Pantoliano, Marcus Chong, Julian Arahanga, Matt Doran e Belinda McClory, que interpretam respectivamente os personagens Neo, Morpheus, Trinity, Agent Smith, Oracle, Cypher, Tank, Apoc e Mouse.'

In [11]:
agent_chain.run("Qual o elenco do filme Star Wars?")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes

Action: Elenco do filme

Action Input: Star Wars

Observation: Elenco do filme Star Wars
: [{'name': 'Mark Hamill', 'character': 'Luke Skywalker / Dobbu Scay (voice)'}, {'name': 'Carrie Fisher', 'character': 'General Leia Organa'}, {'name': 'Adam Driver', 'character': 'Kylo Ren / Ben Solo'}, {'name': 'Daisy Ridley', 'character': 'Rey'}, {'name': 'John Boyega', 'character': 'Finn'}, {'name': 'Oscar Isaac', 'character': 'Poe Dameron'}, {'name': 'Andy Serkis', 'character': 'Supreme Leader Snoke'}, {'name': "Lupita Nyong'o", 'character': 'Maz Kanata'}, {'name': 'Domhnall Gleeson', 'character': 'General Hux'}, {'name': 'Anthony Daniels', 'character': 'C-3PO'}]
Thought:```
Thought: Do I need to use a tool? No

AI: O elenco do filme Star Wars é composto por Mark Hamill, Carrie Fisher, Adam Driver, Daisy Ridley, John Boyega, Oscar Isaac, Andy Serkis, Lupita Nyong'o, Domhnall Gleeson e Anthony Daniels, que interp

"O elenco do filme Star Wars é composto por Mark Hamill, Carrie Fisher, Adam Driver, Daisy Ridley, John Boyega, Oscar Isaac, Andy Serkis, Lupita Nyong'o, Domhnall Gleeson e Anthony Daniels, que interpretam respectivamente os personagens Luke Skywalker / Dobbu Scay (voice), General Leia Organa, Kylo Ren / Ben Solo, Rey, Finn, Poe Dameron, Supreme Leader Snoke, Maz Kanata, General Hux e C-3PO.\n```"

In [12]:
agent_chain.run("Quais são os filmes populares dessa semana?")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes

Action: Filmes populares

Action Input: week

Observation: Os filmes populares no período escolhido são: ['Godzilla e Kong: O Novo Império', 'Ghostbusters: Apocalipse de Gelo', 'Duna: Parte Dois', 'Planeta dos Macacos: O Reinado', 'Godzilla Minus One', 'Abigail', 'The Ministry of Ungentlemanly Warfare', 'Uma Ideia de Você', 'Fúria Primitiva', 'Duna', 'Rivais', 'Furiosa: Uma Saga Mad Max', 'Rebel Moon - Parte 2: A Marcadora de Cicatrizes', 'Kung Fu Panda 4', 'Garfield - Fora de Casa', 'A Batalha do Biscoito Pop-Tart', 'Guerra Civil', 'Amigos Imaginários', 'Exhuma', 'A Mãe da Noiva']
Thought:```
Thought: Do I need to use a tool? Yes

Action: Filmes populares

Action Input: week

Observation: Os filmes populares no período escolhido são: ['Godzilla e Kong: O Novo Império', 'Ghostbusters: Apocalipse de Gelo', 'Duna: Parte Dois', 'Planeta dos Macacos: O Reinado', 'Godzilla Minus One', 'Abigail', 'The Ministry

'Agent stopped due to iteration limit or time limit.'

In [13]:
agent_chain.run("Me fale sobre o filme The Matrix")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes

Action: Pesquisa de filmes

Action Input: The Matrix

Observation: {'adult': False, 'backdrop_path': '/icmmSD4vTTDKOq2vvdulafOGw93.jpg', 'genre_ids': [28, 878], 'id': 603, 'original_language': 'en', 'original_title': 'The Matrix', 'overview': 'Em um futuro próximo, Thomas Anderson (Keanu Reeves), um jovem programador de computador que mora em um cubículo escuro, é atormentado por estranhos pesadelos nos quais encontra-se conectado por cabos e contra sua vontade, em um imenso sistema de computadores do futuro. Em todas essas ocasiões, acorda gritando no exato momento em que os eletrodos estão para penetrar em seu cérebro. À medida que o sonho se repete, Anderson começa a ter dúvidas sobre a realidade. Por meio do encontro com os misteriosos Morpheus (Laurence Fishburne) e Trinity (Carrie-Anne Moss), Thomas descobre que é, assim como outras pessoas, vítima do Matrix, um sistema inteligente e artificial que

'O filme The Matrix foi lançado em 31 de março de 1999. O elenco do filme The Matrix é composto por Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, Gloria Foster, Joe Pantoliano, Marcus Chong, Julian Arahanga, Matt Doran e Belinda McClory, que interpretam respectivamente os personagens Neo, Morpheus, Trinity, Agent Smith, Oracle, Cypher, Tank, Apoc e Mouse.'

In [15]:
agent_chain.run("Me dê a sinopse do filme The Matrix")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes

Action: Pesquisa de filmes

Action Input: The Matrix

Observation: {'adult': False, 'backdrop_path': '/icmmSD4vTTDKOq2vvdulafOGw93.jpg', 'genre_ids': [28, 878], 'id': 603, 'original_language': 'en', 'original_title': 'The Matrix', 'overview': 'Em um futuro próximo, Thomas Anderson (Keanu Reeves), um jovem programador de computador que mora em um cubículo escuro, é atormentado por estranhos pesadelos nos quais encontra-se conectado por cabos e contra sua vontade, em um imenso sistema de computadores do futuro. Em todas essas ocasiões, acorda gritando no exato momento em que os eletrodos estão para penetrar em seu cérebro. À medida que o sonho se repete, Anderson começa a ter dúvidas sobre a realidade. Por meio do encontro com os misteriosos Morpheus (Laurence Fishburne) e Trinity (Carrie-Anne Moss), Thomas descobre que é, assim como outras pessoas, vítima do Matrix, um sistema inteligente e artificial que man

'A sinopse do filme The Matrix é a seguinte: Em um futuro próximo, Thomas Anderson (Keanu Reeves), um jovem programador de computador que mora em um cubículo escuro, é atormentado por estranhos pesadelos nos quais encontra-se conectado por cabos e contra sua vontade, em um imenso sistema de computadores do futuro. Em todas essas ocasiões, acorda gritando no exato momento em que os eletrodos estão para penetrar em seu cérebro. À medida que o sonho se repete, Anderson começa a ter dúvidas sobre a realidade. Por meio do encontro com os misteriosos Morpheus (Laurence Fishburne) e Trinity (Carrie-Anne Moss), Thomas descobre que é, assim como outras pessoas, vítima do Matrix, um sistema inteligente e artificial que manipula a mente das pessoas, criando a ilusão de um mundo real enquanto usa os cérebros e corpos dos indivíduos para produzir energia. Morpheus, entretanto, está convencido de que Thomas é Neo, o aguardado messias capaz de enfrentar o Matrix e conduzir as pessoas de volta à reali